Microsoft Teams Name - Tanvi Ajay Gunjal, Ankita Behura Name - Tanvi Ajay Gunjal Matriculation Number - 7002984 Name - Ankita Behura Matriculation Number - 7002639 Tutor - Redion Xhepa

In [ ]:
"""multilayer_perceptron.py

A Multilayer Perceptron implementation example using Pytorch.
This example does handwritten digit recognition using the MNIST database.
(http://yann.lecun.com/exdb/mnist/)

"""

import torch
import torch.nn as nn
import torchvision
from torchvision import models
from torchvision import datasets
import torchvision.transforms as transforms
import torch.nn.functional as F

from torch.autograd import Variable
from scipy.ndimage.interpolation import shift

In [ ]:
# Network hyperparameters
input_size = 28*28
hidden_layer_size= 512
output_size = 10
num_epochs = 2
batch_size = 100
learning_rate = 1e-3
learning_rate_decay = 0.8
num_workers = 0


Add random noise (or any distribution that you think might be useful). Comment
on the amount of noise and how it affects the results.

In [ ]:
# Add random Noise to data
def add_noise(img):
    noise = torch.randn(img.size()) * 0.2
    noisy_img = img + noise
    return noisy_img


Shift the images in all four directions by the given.

In [ ]:
# Method to shift the image by given dimension
train_transform=transforms.Compose([
    transforms.RandomApply([transforms.RandomAffine(degrees=(0, 0), 
                                  translate=(0.10, 0.10))], p=0.20),
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])


In [ ]:
# Load MNIST train and test datasets 
def load_dataset():
    train_data = datasets.MNIST(root='data', train=True,
                                   download=True, transform=train_transform) # Added Data augmentation transform defined above
    test_data = datasets.MNIST(root='data', train=False,
                                  download=True, transform=transforms.ToTensor())
    
    test_size = len(test_data)
    
    
    train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
    num_workers=num_workers)
    test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, 
    num_workers=num_workers)
    return train_loader,test_loader,test_size

train_loader,test_loader,test_size = load_dataset()


In [ ]:
# function to update learning rate
def update_lr(optimizer, lr):
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr


In [ ]:
# Create model 
class MLP(nn.Module):
    
    def __init__(self):
        super(MLP, self).__init__()
        
        # Input layer
        self.fc1 = nn.Linear(input_size, hidden_layer_size)
        
        # Hidden layer
        self.fc2 = nn.Linear(hidden_layer_size, hidden_layer_size)
        
        # Output layer
        self.fc3 = nn.Linear(hidden_layer_size, output_size)

    def forward(self, x):

        x = x.view(-1, input_size)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
# Initialize model
model = MLP()
print(model)


# Specify loss function
criterion = nn.CrossEntropyLoss()

# Specify optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# Train
model.train()
total_step = len(train_loader)

for epoch in range(num_epochs):

    # Loading each input batch
    for i, (images, labels) in enumerate(train_loader):
               
        # Add some noise to the image
        noisy_img = add_noise(images)

        # Outputs after forward pass
        outputs = model(noisy_img)       
        
         # Calculate loss
        loss = criterion(outputs, labels)

        # Backprop to update model parameters 
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 50 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

    # Update learning rate for next epoch       
    learning_rate *= learning_rate_decay
    update_lr(optimizer, learning_rate)

MLP(
  (fc1): Linear(in_features=784, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=10, bias=True)
)
Epoch [1/2], Step [50/600], Loss: 2.3148
Epoch [1/2], Step [100/600], Loss: 2.2659
Epoch [1/2], Step [150/600], Loss: 2.2591
Epoch [1/2], Step [200/600], Loss: 2.2625
Epoch [1/2], Step [250/600], Loss: 2.2309
Epoch [1/2], Step [300/600], Loss: 2.2141
Epoch [1/2], Step [350/600], Loss: 2.1962
Epoch [1/2], Step [400/600], Loss: 2.1836
Epoch [1/2], Step [450/600], Loss: 2.1538
Epoch [1/2], Step [500/600], Loss: 2.1365
Epoch [1/2], Step [550/600], Loss: 2.1080
Epoch [1/2], Step [600/600], Loss: 2.1021
Epoch [2/2], Step [50/600], Loss: 2.0675
Epoch [2/2], Step [100/600], Loss: 2.0079
Epoch [2/2], Step [150/600], Loss: 2.0303
Epoch [2/2], Step [200/600], Loss: 2.0352
Epoch [2/2], Step [250/600], Loss: 1.9938
Epoch [2/2], Step [300/600], Loss: 1.9790
Epoch [2/2], Step [350/600], Loss: 1.9301
Epoch [2/2],

In [ ]:
# Test
model.eval()
correct=0
for images, labels in test_loader:

    # Compute predicted outputs (forward pass)
    output = model(images)

    # Calculate loss
    loss = criterion(output, labels)

    # Convert output probabilities to predicted class
    _, pred = torch.max(output, 1)

    # compare predictions to true labels
    correct += (pred == labels).sum().item()
    

# Test accuracy
print('Accuracy of the MLP on {} test images: {} %'.format(test_size, 100 * (correct / test_size)))

Accuracy of the MLP on 10000 test images: 71.37 %


Running the above model resulted in accuracy of 49% . Adding some random noise and Random Affine transform on our train data
(data augmentation) gave us loss value as low as 1.8275, which shows some improvement.